In [2]:
#
# Import Libraries
#

import os
import pandas as pd
import pickle
import sys
import networkx as nx
import numpy as np
from scipy.stats import poisson

from functools import reduce

from plotly import graph_objects as go
from plotly.subplots import make_subplots
import plotly
palette = plotly.colors.qualitative.Plotly


utilsPath = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\utils'
if utilsPath not in sys.path:
    sys.path.append(utilsPath)

In [3]:
#
# Paths & Constants
#

corr_path = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Analysis\02-Correlations\PESA\corr.pkl'
pval_path = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Analysis\02-Correlations\PESA\pvals.pkl'

xm_path = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\PESA\WorkingFiles\Xm_norm.tsv'
xq_path = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\PESA\WorkingFiles\Xq_minus_X_norm.tsv'

m2i_path = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Metabolomics\PESA\WorkingFiles\f2i.tsv'
q2i_path = r'S:\U_Proteomica\UNIDAD\software\MacrosRafa\data\Metabolomics\PESA_Integromics\Data\Proteomics\PESA\WorkingFiles\q2info.tsv'

In [4]:
xq = pd.read_csv(xq_path, sep='\t', index_col='Seqn')
xm = pd.read_csv(xm_path, sep='\t', index_col='Seqn')

q2i = pd.read_csv(q2i_path, sep='\t', index_col='fid')
m2i = pd.read_csv(m2i_path, sep='\t', index_col='fid')

In [5]:
#
# Read raw data from which the graph will be built
#

with open(corr_path, 'rb') as f:
    corr, corrSL = pickle.load(f)

with open(pval_path, 'rb') as f:
    pv, adpv = pickle.load(f)

Analizar orden y tamaño del grafo usando diferentes tipos de correlaciones y umbrales de FDR

In [6]:
#
# 
#

ctypes = ['rpc', 'psk']

pvrange = np.arange(0,0.2,0.05)

plot = {}

for ctype in ctypes:
    plot[ctype] = {
        'qq': {'nodes':[], 'edges':[]},
        'mm': {'nodes':[], 'edges':[]}, 
    }
    for pv in pvrange:

        qqc = getattr(corr, ctype).qq.dc.copy()
        qqp = adpv[ctype].qq.dc.copy()
        qqg = nx.from_pandas_adjacency(qqc[qqp<=pv].fillna(0))
        plot[ctype]['qq']['nodes'].append((np.array(list(dict(nx.degree(qqg)).values()))>0).sum())
        plot[ctype]['qq']['edges'].append(qqg.size())

        mmc = getattr(corr, ctype).mm.dc.copy()
        mmp = adpv[ctype].mm.dc.copy()
        mmg = nx.from_pandas_adjacency(mmc[mmp<=pv].fillna(0))
        plot[ctype]['mm']['nodes'].append((np.array(list(dict(nx.degree(mmg)).values()))>0).sum())
        plot[ctype]['mm']['edges'].append(mmg.size())


ctypes2 = ['rcca', 'psk', 'cca']

for ctype in ctypes2:
    if ctype not in plot.keys(): plot[ctype]={}
    plot[ctype]['qm'] = {
        'nodes':[], 'edges':[],
    }
    for pv in pvrange:

        qmc = getattr(corr, ctype).qm.dc.copy()
        qmp = adpv[ctype].qm.dc.copy()
        qmg = qmc[qmp<pv].fillna(0)
        qmg = pd.concat([
            pd.DataFrame(columns=qmg.index, index=qmg.index).fillna(0).join(qmg),
            qmg.T.join(pd.DataFrame(columns=qmg.columns, index=qmg.columns).fillna(0))
        ])
        qmg = nx.from_pandas_adjacency(qmg)
        plot[ctype]['qm']['nodes'].append((np.array(list(dict(nx.degree(qmg)).values()))>0).sum())
        plot[ctype]['qm']['edges'].append(qmg.size())

In [7]:
from plotly.subplots import make_subplots
from plotly import graph_objects as go
import plotly.express as px

palette = px.colors.qualitative.Plotly

file = 'Plots/FDR_CorrType.html'
if os.path.exists(file):
    os.remove(file)

fig = make_subplots(rows=3, cols=2, subplot_titles=['qq - Nodes', 'qq - Edges','mm - Nodes', 'mm - Edges', 'qm - Nodes', 'qm - Edges'])
plot['psk']['qq']['nodes']

for n,ctype in enumerate(ctypes):
    fig.add_trace(go.Scatter(
        x=pvrange,
        y=plot[ctype]['qq']['nodes'],
        name=ctype, marker_color=palette[n], legendgroup=n
    ), row=1, col=1)
    fig.add_trace(go.Scatter(
        x=pvrange,
        y=plot[ctype]['qq']['edges'],
        name=ctype, marker_color=palette[n], legendgroup=n
    ), row=1, col=2)
    fig.add_trace(go.Scatter(
        x=pvrange,
        y=plot[ctype]['mm']['nodes'],
        name=ctype, marker_color=palette[n], legendgroup=n
    ), row=2, col=1)
    fig.add_trace(go.Scatter(
        x=pvrange,
        y=plot[ctype]['mm']['edges'],
        name=ctype, marker_color=palette[n], legendgroup=n
    ), row=2, col=2)

for n,ctype in enumerate(ctypes2):
    fig.add_trace(go.Scatter(
        x=pvrange,
        y=plot[ctype]['qm']['nodes'],
        name=ctype, marker_color=palette[n], legendgroup=n
    ), row=3, col=1)
    fig.add_trace(go.Scatter(
        x=pvrange,
        y=plot[ctype]['qm']['edges'],
        name=ctype, marker_color=palette[n], legendgroup=n
    ), row=3, col=2)

fig.update_layout(title='Number of nodes and edges per correlation type', width=700, height=500)
#fig.show()

with open(file, 'a') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn', default_height='50%', default_width='80%'))


Construcción de grafo usando Graphical Lasso y rCCA con FDR <0.05

In [12]:
# Checkpoint
qqc,qqp,qqg,mmc,mmp,mmg,qmc,qmp,qmg = pickle.load(open('sourceData.pkl', 'rb'))

In [8]:
pv = 0.05

qqc = corr.rpc.qq.dc.copy()
qqp = adpv.rpc.qq.dc.copy()
qqg = nx.from_pandas_adjacency(qqc[qqp<=pv].fillna(0))

mmc = corr.rpc.mm.dc.copy()
mmp = adpv.rpc.mm.dc.copy()
mmg = nx.from_pandas_adjacency(mmc[mmp<=pv].fillna(0))

qmc = corr.rcca.qm.dc.copy()
qmp = adpv.rcca.qm.dc.copy()
qmg = qmc[qmp<pv].fillna(0)
qmg = pd.concat([
    pd.DataFrame(columns=qmg.index, index=qmg.index).fillna(0).join(qmg),
    qmg.T.join(pd.DataFrame(columns=qmg.columns, index=qmg.columns).fillna(0))
])
qmg = nx.from_pandas_adjacency(qmg)

In [10]:
# Checkpoint
pickle.dump([qqc,qqp,qqg,mmc,mmp,mmg,qmc,qmp,qmg], open('sourceData.pkl', 'wb'))

In [13]:
#
# Generate random graphs
#
n = 100

qqA = [nx.gnm_random_graph(qqg.order(), qqg.size()) for i in range(n)]
mmA = [nx.gnm_random_graph(mmg.order(), mmg.size()) for i in range(n)]

In [228]:
#
# Basic graph characteristics
#

G = qqg
GA = qqA

for G, GA, name in [(qqg, qqA, 'Proteomics'), (mmg, mmA, 'Metabolomics')]:

    print()
    print(name)
    print()

    print(f'Order: {G.order()}')
    print(f'Size: {G.size()}')
    print(f'Density: {round(nx.density(G), 5)}')

    i = [nx.average_clustering(i) for i in GA]
    print(f'Average clustering: {round(nx.average_clustering(G), 5)} (random graph: mean = {round(np.mean(i),5)}, std = {round(np.std(i),5)})')
    print(f'Average degree: {round(np.mean(list(dict(G.degree).values())),5)}')

    i = [len(list(nx.connected_components(i))) for i in GA]
    print(f'Number of connected components: {round(len(list(nx.connected_components(G))), 5)} (random graph: mean = {round(np.mean(i), 5)}, std = {round(np.std(i), 5)})')

    i = [len([i for i in nx.connected_components(g) if len(i)>1]) for g in GA]
    print(f'Number of connected components (order g.t. 1): {round(len([i for i in nx.connected_components(G) if len(i)>1]), 5)} (random graph: mean = {round(np.mean(i),5)}, std = {round(np.std(i),5)})')

fig = make_subplots(rows=1, cols=2, subplot_titles=['Proteomics', 'Metabolomics'])

plot = nx.degree_histogram(qqg)
fig.add_trace(go.Bar(
    x=np.arange(0,len(plot)),
    y=plot,
    offsetgroup=1, marker_color=palette[0]
), row=1,col=1)
fig.add_trace(go.Bar(
    x=np.arange(0,len(plot)),
    y=G.order()*poisson.pmf(np.arange(0, len(plot)),2*G.size()/G.order()),
    offsetgroup=1, marker_color='black', opacity=0.2
), row=1,col=1)


plot = nx.degree_histogram(mmg)
fig.add_trace(go.Bar(
    x=np.arange(0,len(plot)),
    y=plot, 
    offsetgroup=1, marker_color=palette[1]
), row=1,col=2)
fig.add_trace(go.Bar(
    x=np.arange(0,len(plot)),
    y=G.order()*poisson.pmf(np.arange(0, len(plot)),2*G.size()/G.order()),
    offsetgroup=1, marker_color='black', opacity=0.2
), row=1,col=2)

fig.update_layout(bargap=0.2, title='Degree Distribution', showlegend=False)


Proteomics

Order: 470
Size: 3098
Density: 0.02811
Average clustering: 0.11233 (random graph: mean = 0.02815, std = 0.00159)
Average degree: 13.18298
Number of connected components: 9 (random graph: mean = 1.0, std = 0.0)
Number of connected components (order g.t. 1): 1 (random graph: mean = 1.0, std = 0.0)

Metabolomics

Order: 1826
Size: 2904
Density: 0.00174
Average clustering: 0.19416 (random graph: mean = 0.00142, std = 0.00078)
Average degree: 3.18072
Number of connected components: 1215 (random graph: mean = 83.95, std = 8.25273)
Number of connected components (order g.t. 1): 124 (random graph: mean = 7.05, std = 2.72534)


In [105]:
#
# Number of connected components with different sizes
#

G = mmg

fig = make_subplots(rows=1, cols=2, subplot_titles=['Proteomics', 'Metabolomics'])#go.Figure()
for n,G in enumerate([qqg, mmg]):
    plot = pd.Series([len(i) for i in nx.connected_components(G)]).value_counts().sort_index()
    fig.add_trace(go.Bar(
        x = [str(i) for i in plot.index],
        y = plot.values,
        width=0.5,
        text=plot.values, textposition='auto', 
    ), row=1, col=n+1)

fig.update_layout(bargap=1)
fig.update_layout(title=f'Number of connected components with different order', showlegend=False)
fig.show()

In [104]:
#
# Number of cliques with diferent orders
#

fig = make_subplots(rows=1, cols=2, subplot_titles=['Proteomics', 'Metabolomics'])#go.Figure()

for n,G in enumerate([qqg, mmg]):
    plot = pd.Series([len(i) for i in nx.find_cliques(G)]).value_counts().sort_index()
    fig.add_trace(go.Bar(
        x = [str(i) for i in plot.index],
        y = plot.values,
        width=0.5,
        text=plot.values, textposition='auto', 
    ), row=1, col=n+1)

    fig.update_layout(bargap=1)
fig.update_layout(title=f'Number of cliques with different order', showlegend=False)
fig.show()

In [378]:
#
# Build complete graph adding omic type to each node
#

g = nx.compose_all([qqg, mmg, qmg])

attr = {}
_ = [attr.update({i:'q'}) for i in q2i.index]
_ = [attr.update({i:'m'}) for i in m2i.index]
nx.set_node_attributes(g, attr, 'group')
_ = [attr.update({i:'blue'}) for i in q2i.index]
_ = [attr.update({i:'red'}) for i in m2i.index]
nx.set_node_attributes(g, attr, 'color')

OBTAIN CLUSTERS/COMMUNITIES

In [819]:
#
# Create communities (clusters) 
# Louvain & Leiden; Finally we use Leiden

# comm = nx_comm.louvain_communities(g, resolution=1, seed=0)

In [953]:
import leidenalg
import igraph as ig

G= mmg
n = 4
thr=10


def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

def get_communities(G, n=5, thr=5):
    comm = [
        list(i.to_networkx().nodes)
        for i in leidenalg.find_partition(
                ig.Graph.from_networkx(G), leidenalg.ModularityVertexPartition, 
                n_iterations=-1, seed=-1
            ).subgraphs()
    ]

    for i in range(n):
        commi = [
        list(i.to_networkx().nodes)
        for i in leidenalg.find_partition(
                ig.Graph.from_networkx(G), leidenalg.ModularityVertexPartition, 
                n_iterations=-1, seed=i
            ).subgraphs()
    ]
        comm = [
        np.intersect1d(j, commi[np.argmax([jaccard(j,k) for k in commi])]) for j in comm
        ]

        comm = [j for j in comm if nx.is_connected(G.subgraph(j)) and len(j)>thr]
    
    return comm


qqcom = get_communities(qqg, n=n, thr=10)
mmcom = get_communities(mmg, n=n, thr=10)

In [954]:
print(f'Proteomics communities: {len(qqcom)}')
print(f'Metabolomics communities: {len(mmcom)}')

Proteomics communities: 6
Metabolomics communities: 13


In [957]:
get_communities(qqA[10], n=n, thr=thr)


[]

In [921]:
atable = m2i.loc[mmcom[0]][['ID', 'TP_ID']]
atable = q2i.loc[qqcom[4]]

GRAPH VISUALIZATION

In [840]:
from pyvis.network import Network

nt = Network(height='500px', width='100%')
nt.repulsion()
nt.force_atlas_2based()
nt.from_nx(g.subgraph(mmcom[13]))
#nt.from_nx(qqA[0].subgraph(list(get_communities(qqA[0], n=n)[0])))
nt.show_buttons(filter_='physics')
nt.show('nx.html', notebook=False)

AssertionError: 

STRING ENRICHMENT

In [735]:
from STRING import Uniprot2String, FunctionalEnrichment
background = Uniprot2String(
    q2i.index.tolist()
)

background = [i[1] for i in background]

In [916]:
my_genes = Uniprot2String(
    qqcom[0]
)
my_genes = [i[1] for i in my_genes]
data = FunctionalEnrichment(my_genes, background, species=9606)

from io import StringIO

data = pd.read_csv(
    StringIO(data),
    sep='\t'
)

col = ['category', 'number_of_genes', 'number_of_genes_in_background', 'fdr', 'description']
categories = ['Process', 'KEGG', 'Component']
atable = data[np.isin(data['category'], categories)].sort_values('fdr').loc[:, col]#.head(15)
atable


,category,number_of_genes,number_of_genes_in_background,fdr,description
